# Notebook de treinamento
Bibliotecas

In [1]:
import ipywidgets as widgets
import numpy as np
import pandas as pd

import velha as jv

from collections import OrderedDict

In [2]:
versao = "-v0.0"
sobrescreve = False

In [3]:
niveis_treinamento = OrderedDict({})
labels = ['muito fraco', 'fraco', 'medio', 'forte', 'muito forte']
menor_potencia = 2
maior_potencia = menor_potencia + len(labels)
rodadas = 10 ** np.arange(menor_potencia, maior_potencia)
for indice, label in enumerate(labels):
    niveis_treinamento[label] = {'rodadas': rodadas[indice]}
    niveis_treinamento[label]['progresso'] = widgets.FloatProgress(
        value=0.0,
        min=0.0,
        max=1.0,
        description=f'{label}:',
        bar_style='success',
        style={'bar_color': 'black'},
        orientation='horizontal'
    )

In [4]:
# Treinamento de dois jogadores por um número de rodadas

for nivel in niveis_treinamento:
    rodadas_treinamento = niveis_treinamento[nivel]['rodadas']
    print(f'Iniciando treinamento com {rodadas_treinamento} rodadas... ({nivel})')
    verifica = rodadas_treinamento / 10
    politica_X = f"X{rodadas_treinamento}{versao}"
    politica_O = f"O{rodadas_treinamento}{versao}"

    if sobrescreve or (not jv.existe_politica(politica_X) or not jv.existe_politica(politica_O)):
        primeiro = jv.Maquina(politica_X)
        segundo = jv.Maquina(politica_O)
        treino = jv.jogoDaVelha(primeiro, segundo)
        treino.treinamento(rodadas_treinamento, verifica=verifica, progresso=niveis_treinamento[nivel]['progresso'])
        print(f"Salvando políticas para {rodadas_treinamento} rodadas")
        print(f'Salvando política: {primeiro.nome}')
        primeiro.salva_politica()
        print(f'Salvando política: {segundo.nome}')
        segundo.salva_politica()
    else:
        print(f'Políticas {politica_O} e {politica_X} já existem')

Iniciando treinamento com 100 rodadas... (muito fraco)
Políticas O100-v0.0 e X100-v0.0 já existem
Iniciando treinamento com 1000 rodadas... (fraco)
Políticas O1000-v0.0 e X1000-v0.0 já existem
Iniciando treinamento com 10000 rodadas... (medio)
Políticas O10000-v0.0 e X10000-v0.0 já existem
Iniciando treinamento com 100000 rodadas... (forte)
Políticas O100000-v0.0 e X100000-v0.0 já existem
Iniciando treinamento com 1000000 rodadas... (muito forte)
Políticas O1000000-v0.0 e X1000000-v0.0 já existem


# Combinando políticas

In [5]:
for nivel in niveis_treinamento:
    politica_combinada = f"XO{niveis_treinamento[nivel]['rodadas']}{versao}"
    if sobrescreve or not jv.existe_politica(politica_combinada):
        primeiro = jv.Maquina("velhaRLX", taxa_exploracao=0.0)
        primeiro.carrega_politica(f"X{niveis_treinamento[nivel]['rodadas']}{versao}")
        segundo = jv.Maquina("velhaRLO", taxa_exploracao=0.0)
        segundo.carrega_politica(f"O{niveis_treinamento[nivel]['rodadas']}{versao}")

        primeiro.combina_e_salva_politica(segundo, politica_combinada)
        print(f"Salvando política {politica_combinada}!!!")
    else:
        print(f"Arquivo de política {politica_combinada} já existe!")

Arquivo de política XO100-v0.0 já existe!
Arquivo de política XO1000-v0.0 já existe!
Arquivo de política XO10000-v0.0 já existe!
Arquivo de política XO100000-v0.0 já existe!
Arquivo de política XO1000000-v0.0 já existe!


# Jogando todas políticas contra todas

In [6]:
num_partidas = widgets.IntSlider(
    value=10000,
    min=1000,
    max=50000,
    step = 100,
    description='Rodadas: '
)
num_partidas

IntSlider(value=10000, description='Rodadas: ', max=50000, min=1000, step=100)

In [7]:
resultados = {}
for nivel_X in niveis_treinamento:
    resultados['X ' + nivel_X] = {}
    for nivel_O in niveis_treinamento:
        rodadas_X = niveis_treinamento[nivel_X]['rodadas']
        rodadas_O = niveis_treinamento[nivel_O]['rodadas']
        X = jv.Maquina(f"X-{rodadas_X}", taxa_exploracao=0.0, limite_exploracao=0.0)
        X.carrega_politica(f"X{rodadas_X}{versao}")
        O = jv.Maquina(f"O-{rodadas_O}", taxa_exploracao=0.0, limite_exploracao=0.0)
        O.carrega_politica(f"O{rodadas_O}{versao}")

        simulacao = jv.jogoDaVelha(X, O)
        total, tabuleiros = simulacao.simulacao(partidas=num_partidas.value)
        resultados['X ' + nivel_X]['O ' + nivel_O] = total
        print(f"{X.nome}, {O.nome}")
        print(total)
        
        print()

X-100, O-100
Counter({'O-100': 4012, 'X-100': 3803, 'Velha': 2185})

X-100, O-1000
Counter({'Velha': 5972, 'X-100': 2353, 'O-1000': 1675})

X-100, O-10000
Counter({'Velha': 5295, 'O-10000': 4705})

X-100, O-100000
Counter({'O-100000': 10000})

X-100, O-1000000
Counter({'O-1000000': 10000})

X-1000, O-100
Counter({'X-1000': 5859, 'O-100': 3899, 'Velha': 242})

X-1000, O-1000
Counter({'O-1000': 6600, 'Velha': 2540, 'X-1000': 860})

X-1000, O-10000
Counter({'O-10000': 8349, 'Velha': 1651})

X-1000, O-100000
Counter({'O-100000': 7459, 'Velha': 2541})

X-1000, O-1000000
Counter({'O-1000000': 7494, 'Velha': 2506})

X-10000, O-100
Counter({'X-10000': 7911, 'O-100': 2089})

X-10000, O-1000
Counter({'X-10000': 10000})

X-10000, O-10000
Counter({'Velha': 10000})

X-10000, O-100000
Counter({'O-100000': 10000})

X-10000, O-1000000
Counter({'O-1000000': 10000})

X-100000, O-100
Counter({'X-100000': 10000})

X-100000, O-1000
Counter({'X-100000': 10000})

X-100000, O-10000
Counter({'Velha': 10000})



In [8]:
pd.DataFrame(resultados)

,X muito fraco,X fraco,X medio,X forte,X muito forte
O muito fraco,"{'O-100': 4012, 'X-100': 3803, 'Velha': 2185}","{'X-1000': 5859, 'O-100': 3899, 'Velha': 242}","{'O-100': 2089, 'X-10000': 7911}",{'X-100000': 10000},"{'X-1000000': 8703, 'Velha': 1297}"
O fraco,"{'Velha': 5972, 'X-100': 2353, 'O-1000': 1675}","{'O-1000': 6600, 'X-1000': 860, 'Velha': 2540}",{'X-10000': 10000},{'X-100000': 10000},{'Velha': 10000}
O medio,"{'O-10000': 4705, 'Velha': 5295}","{'O-10000': 8349, 'Velha': 1651}",{'Velha': 10000},{'Velha': 10000},{'Velha': 10000}
O forte,{'O-100000': 10000},"{'Velha': 2541, 'O-100000': 7459}",{'O-100000': 10000},{'Velha': 10000},{'Velha': 10000}
O muito forte,{'O-1000000': 10000},"{'Velha': 2506, 'O-1000000': 7494}",{'O-1000000': 10000},{'Velha': 10000},{'Velha': 10000}


# Consulta à tabela Q de X

In [9]:
niveis_treinamento['forte']

{'rodadas': 100000,
 'progresso': FloatProgress(value=0.0, bar_style='success', description='forte:', max=1.0, style=ProgressStyle(bar_color='black'))}

In [10]:
primeiro = jv.Maquina("velhaRL", taxa_exploracao=0.0)
primeiro.carrega_politica(f"X{niveis_treinamento['forte']['rodadas']}{versao}")

for tabuleiro in [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 2, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 2]]:
    estado = jv.gera_hash_tabuleiro(np.array(tabuleiro))
    print(f"Estado: {estado}")
    jv.mostra_tabuleiro(tabuleiro)
    print(f"Tabela Q: {primeiro.q[estado]}")
    print(f"Lance da política = {max(primeiro.q[estado], key=primeiro.q[estado].get)}\n")

Estado: [0 0 0 0 0 0 0 0 0]
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
Tabela Q: {0: -1.5357601892540917, 1: -1.1183232135161614, 2: -0.11031997430357898, 3: -1.531206709438024, 4: -0.8751453329304233, 5: -1.0610920215394215, 6: -1.2297982738053046, 7: -1.6106207906101686, 8: 0.4244108112738462}
Lance da política = 8

Estado: [0 2 0 0 1 0 0 0 0]
-------------
|   | O |   | 
-------------
|   | X |   | 
-------------
|   |   |   | 
-------------
Tabela Q: {0: 1.06284374906368, 2: 4.241240751477566, 3: 3.818752694991051, 5: 2.338991131473389, 6: 3.5212294406884013, 7: -0.6722996458108266, 8: 3.962666251997919}
Lance da política = 2

Estado: [0 0 0 0 0 0 0 1 2]
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
|   | X | O | 
-------------
Tabela Q: {0: -0.6544000000000001, 1: -0.8257966720000001, 2: 0.7111690640501058, 3: -0.44386778335345106, 4: -0.5017702358946827, 5: -0.5038255591977637, 6: -0.6833368165406

# Consulta à tabela Q de O

In [11]:
segundo = jv.Maquina("velhaRLO", taxa_exploracao=0.0)
segundo.carrega_politica(f"O{niveis_treinamento['forte']['rodadas']}{versao}")

for tabuleiro in [[0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 2, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1, 2], [0, 0, 0, 0, 0, 0, 0, 0, 1]]:
    estado = jv.gera_hash_tabuleiro(np.array(tabuleiro))
    print(f"Estado: {estado}")
    jv.mostra_tabuleiro(tabuleiro)
    print(f"Tabela Q: {segundo.q[estado]}")
    print(f"Lance da política = {max(segundo.q[estado], key=segundo.q[estado].get)}\n")

Estado: [0 0 0 0 1 0 0 0 0]
-------------
|   |   |   | 
-------------
|   | X |   | 
-------------
|   |   |   | 
-------------
Tabela Q: {0: 1.1101455525691746, 1: -2.617115504852277, 2: 1.0946481252921874, 3: -2.8202382190093105, 5: -3.0371218087031875, 6: 2.504589511065591, 7: -2.527549520764842, 8: 1.1780104649258123}
Lance da política = 6

Estado: [1 0 0 0 2 0 0 0 1]
-------------
| X |   |   | 
-------------
|   | O |   | 
-------------
|   |   | X | 
-------------
Tabela Q: {1: 1.650982704176292, 2: -1.3835960352282803, 3: 1.4150649137314242, 5: 1.576321054258759, 6: -2.390215703628281, 7: 3.317887896847863}
Lance da política = 7

Estado: [0 1 0 0 0 0 0 1 2]
-------------
|   | X |   | 
-------------
|   |   |   | 
-------------
|   | X | O | 
-------------
Tabela Q: {0: -0.08784000000000003, 2: 0.0, 3: 0.0, 4: 3.9319161662366993, 5: 0.0, 6: -0.22000000000000003}
Lance da política = 4

Estado: [0 0 0 0 0 0 0 0 1]
-------------
|   |   |   | 
-------------
|   |   |   | 
-------

# Tabela Q completa para X

[Pular para partidas](#Jogando-algumas-partidas)

In [12]:
primeiro.q

{'[0 0 0 0 0 0 0 0 0]': {0: -1.5357601892540917,
  1: -1.1183232135161614,
  2: -0.11031997430357898,
  3: -1.531206709438024,
  4: -0.8751453329304233,
  5: -1.0610920215394215,
  6: -1.2297982738053046,
  7: -1.6106207906101686,
  8: 0.4244108112738462},
 '[0 0 0 0 0 2 0 1 0]': {0: -0.253626688,
  1: 0.0,
  2: 0.18411494400000006,
  3: -0.03199999999999997,
  4: 4.003297753114697,
  6: 0.0,
  8: 0.39891188224},
 '[0 0 0 1 0 2 2 1 0]': {0: 0.0,
  1: 0.0,
  2: 0.0,
  4: 0.0,
  8: 0.31000000000000005},
 '[0 0 2 1 0 2 2 1 1]': {0: 0.0, 1: 0.0, 4: 0.31000000000000005},
 '[0 2 2 1 1 2 2 1 1]': {0: 2.9449889814468557},
 '[0 0 0 0 2 0 0 1 0]': {0: -1.5283894407325787,
  1: -3.498329787818205,
  2: -1.5803704738177413,
  3: -1.5610609072385957,
  5: -1.5954082349972416,
  6: -1.2995365716341665,
  8: -1.7911461386595298},
 '[0 0 0 1 2 2 0 1 0]': {0: -0.22502215520089988,
  1: -0.27999999999999997,
  2: -0.14004433585743128,
  6: 3.764831731760363,
  8: 0.0},
 '[0 0 1 1 2 2 2 1 0]': {0: -0.948